In [9]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import pprint

#Maths
import numpy as np
from numpy import arange, amin, amax, average

#Plot
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import (NavigationToolbar2QT as NavigationToolbar)
from matplotlib.widgets import Cursor, RectangleSelector
import matplotlib.pyplot as plt
#%matplotlib inline
#%matplotlib notebook
%matplotlib widget
plt.rcParams.update({'figure.max_open_warning': 0})

#Astro / Physics
from astropy.io import fits
import astropy.units as u
from astropy.modeling import models
from astropy.visualization import quantity_support
quantity_support()# for getting units on the axes below  
#imports wcs 
#Specutils always maintains a 
#WCS object whether it is passed explicitly by the user,
# or is created dynamically by specutils itself.
import astropy.wcs as fitswcs
from specutils import Spectrum1D, SpectralRegion
from specutils.analysis import snr
from specutils.manipulation import noise_region_uncertainty
from specutils.manipulation import extract_region
from specutils.fitting import fit_generic_continuum
from specutils.fitting import find_lines_derivative
from specutils.fitting import fit_lines
from specutils.manipulation import snr_threshold
from specutils.fitting import find_lines_threshold
from specutils.fitting import estimate_line_parameters
from specutils.analysis import centroid, fwhm
from astropy.modeling.models import Lorentz1D
from specutils.analysis import gaussian_sigma_width, gaussian_fwhm, fwhm, fwzi
   

import specutils
import astropy
print('Specutils : ',specutils.__version__)
print('Astropy : ', astropy.__version__)
print('Numpy : ', np.__version__)

#warning suppression for Spectrum not below the threshold signal-to-noise 0.01. 
specutils.conf.do_continuum_function_check = False

Specutils :  1.0
Astropy :  4.0.1.post1
Numpy :  1.18.2


In [6]:
!pwd
!ls .

/home/gnthibault/projects/labelray
EllipsoidEstimation.ipynb   labelray.py        test.png
geometric_mapping.ipynb     spec_viewer.ipynb
HD121996_Calibration_C.fit  test.geojson


In [7]:
######### Spectrum path ###############

spec_path1 = "./HD121996_Calibration_C.fit"


###########  General configuration for plotting ############
show_elements_lines = True
show_second_axis = True



######### Config values #########
#plt theme possible
#['seaborn-whitegrid', 'seaborn-poster', 'classic', 'seaborn-darkgrid', 'seaborn-ticks', 'seaborn-dark-palette', 'bmh', 'seaborn-notebook', 'fivethirtyeight', 'seaborn-talk', 'ggplot', 'seaborn-dark', 'seaborn-muted', 'seaborn-white', 'seaborn-pastel',
#'fast', 'Solarize_Light2', 'seaborn-paper', 'seaborn-colorblind', '_classic_test', 'seaborn', 'seaborn-bright', 'grayscale', 'seaborn-deep', 'dark_background']



#--- PLT Theme---#
plt.style.use(['fast'])
#plt.style.use(['seaborn-darkgrid'])
#plt.style.use('seaborn-whitegrid')

#generic spec configuration
spec_line_width = 0.8
spec_second_line_width = 0.5

#Spec Unique
spec1_title_font_size = 12
spec1_title_color = "black"
spec1_line_color = "blue"
spec1_second_line_color = "deepskyblue"
second_line_ratio = 2

#Spectral lines value & legend
#spectral line top text position
text_pos_height = 1.5

balmer = {6563 : "Halpha",
          4861 : "Hbeta",
          4340 : "Hgamma"}

manual_elem_line = { 5270 : "FeI",
                    5167 : "MgII" }

spectral_line_to_show = {}
spectral_line_to_show.update(balmer)
#spectral_line_to_show.update(manual_elem_line)






In [16]:
def generatePlot(spec):

    #prepare labels
    spectrumTitle = header['OBJNAME'] + ' - ' + header['DATE-OBS'] + ' - ' + str(header['EXPTIME']) +  's. - ' + header['BSS_INST'] + ' - ' + header['OBSERVER']

    fig, ax1 = plt.subplots(figsize=(19, 10))
    #fig, ax1 = plt.subplots(figsize=(1.400, 660), dpi=100)
    
    #--- AX1 ---#
    label1 = header['OBJNAME'] + ' - ' + header['DATE-OBS']
    ax1.plot(spec.spectral_axis * u.AA, spec.flux, color=spec1_line_color, linewidth=spec_line_width, label = label1)
   
    #Plot Title
    ax1.set_title(spectrumTitle, loc='center', fontsize=spec1_title_font_size, fontweight=0, color=spec1_title_color)

    #X axis label
    ax1.set_xlabel(header['CTYPE1'] + ' - ' + header['CUNIT1'], fontdict=None, labelpad=None)

    #Y axis label
    ax1.set_ylabel('Relative Flux', fontdict=None, labelpad=None)

    ax1.grid(color='grey', alpha=0.8, linestyle='-.', linewidth=0.2, axis='both') 
    
    legend_line = header['OBJNAME'] + ' - ' + header['DATE-OBS']
    ax1.legend([label1], loc=('best'))
    
    ax1.set_xlim([min(spec.spectral_axis).value,max(spec.spectral_axis).value])
    ax1.set_ylim([min(spec.flux).value - 0.5 ,max(spec.flux).value + 0.5])
    
    #--- AX2 ---#
    if(show_second_axis):
        ax2 = ax1.twinx()
        ax2.plot(spec.spectral_axis, spec.flux, color=spec1_second_line_color, alpha=0.3, linewidth = spec_second_line_width, label="second")
        ax2.set_ylabel(" ")
        ax2.grid(False) 

        start_x, end_x = ax1.get_xlim()
        start_y, end_y = ax1.get_ylim()
        ####Set limit here for modify second spectrum multiplication 
        ax2.set(ylim=(start_y/second_line_ratio, end_y/second_line_ratio))
     
        
       
    #--- Show spectral lines for each elements ------------------#
    if(show_elements_lines):
        for val in spectral_line_to_show :

            #Get flux value for each line
            current_element_pos = int(round((val-header["CRVAL1"])/header["CDELT1"]))
            spectral_value = spec.spectral_axis[current_element_pos]
            flux_value = spec.flux[current_element_pos] / u.Jy

            text_position = np.round((flux_value + text_pos_height),1)

            #Line configuration
            markerline = ax1.stem([val], [text_position.value], use_line_collection=True, bottom=flux_value.value, markerfmt='D')
            plt.setp(markerline, color='red', linewidth= 0.8, alpha=0.5, linestyle='--')

            #Text configuration
            ax1.annotate(spectral_line_to_show[val], xy=(val,text_position.value), xycoords='data',
            xytext=(-5,12), textcoords='offset points',  rotation=90)

        
        
    plt.savefig(header['OBJNAME'] + '.png', bbox_inches='tight', dpi=100)
    plt.show()
    

In [17]:
#load spectrum file -> change PathFile if needed
file = fits.open(spec_path1)  

specdata = file[0].data
header = file[0].header
print(repr(specdata))
print(repr(header))

file.close() 



#get WCS & flux wit astropy
wcs_data = fitswcs.WCS(header={'CDELT1': header['CDELT1'], 'CRVAL1': header['CRVAL1'],
                               'CUNIT1': header['CUNIT1'], 'CTYPE1': header['CTYPE1'],
                               'CRPIX1': header['CRPIX1']})

flux= specdata * u.Jy

#get uncertainty (subclass of nddata : https://docs.astropy.org/en/stable/api/astropy.nddata.NDData.html#astropy.nddata.NDData)
from astropy.nddata import StdDevUncertainty
uncertainty = StdDevUncertainty(0.2*np.ones(flux.shape))
                                
spec = Spectrum1D(flux=flux, wcs=wcs_data, uncertainty=uncertainty)

#Show Plot
generatePlot(spec)

array([ 1440.5785,  3617.774 ,  5794.969 , ..., 17104.736 , 15585.447 ,
       13664.022 ], dtype=float32)
SIMPLE  =                    T                                                  
BITPIX  =                  -32 / bits per data value                            
NAXIS   =                    1 / Number of data axes                            
NAXIS1  =                10757 / Length of data axis 1                          
EXPTIME =              151.416 / [s] Total observation duration                 
DATE-OBS= '2020-05-20T01:30:00.357' / Date of observation start                 
CCD-TEMP=                  -20 / [deg] Camera sensor temperature                
DATAMIN =   431.73625582924302 / [adu] minimum value for all pixels             
DATAMAX =   2872943.2399797738 / [adu] maximum value for all pixels             
BSS_SITE= 'Lorient' / Observation site                                          
BSS_LONG= '0' / Observation site longitude                                      
BS

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# ----------------------------------------
# Analysis Part

## Methods here

# ----------------------------------------

In [2]:
def get_uncertainty_from_region(spectrum, start_region, stop_region):
    """
    Get uncertainty from a spectral region.
    Select a spectral region without lines
    
    spectrum : spectrum to analyse
    
    start_region : start of spectral region
    
    stop_region : stop of spectral region
    
    return : uncertainty of the spectral region
    """
    noise_region = SpectralRegion(start_region*u.AA, stop_region*u.AA)
    spec_uncertainty = noise_region_uncertainty(spectrum, noise_region)
    return spec_uncertainty



def cut_extremity_spectrum(spec_to_cut, left_size, right_size):
    """
    Cut extremity of spectrum 
    spec_to_cut : le spectre à couper
    
    left_size : taille de coupe du début du spectre à X
    
    right_size : taille de coupe de la fin - Y
    """
    extremity_selection = SpectralRegion((np.min(spec_to_cut.spectral_axis.value) + left_size)*u.Angstrom, (np.max(spec_to_cut.spectral_axis.value) - right_size)*u.Angstrom)
    spectrum_shorted = extract_region(spec_to_cut, extremity_selection)
    uncertainty = StdDevUncertainty(0.2*np.ones(spectrum_shorted.flux.shape))
    spec_cutted = Spectrum1D(flux=spectrum_shorted.flux, spectral_axis = spectrum_shorted.spectral_axis, uncertainty=uncertainty)
    return spec_cutted


def quick_plot_init(spec_to_plot):
    quick_ax = plt.subplots()[1]  
    quick_ax.plot(spec_to_plot.spectral_axis, spec_to_plot.flux)  
    quick_ax.set_xlabel("Wavelength")  
    quick_ax.set_ylabel("Intensity")
    
    
def show_continuum(spec_to_analyze):
    """
    Get continuum for a spectrum
    """
    x = spec_to_analyze.spectral_axis
    y = spec_to_analyze.flux
    
    g1_fit = fit_generic_continuum(spec_to_analyze)
    y_continuum_fitted = g1_fit(x)
    
    fitest = plt.subplots()[1]  
    fitest.plot(x, y)
    fitest.plot(x, y_continuum_fitted)
    fitest.grid(True)


def normalize_spectrum(spec_to_norm):
    
    x_2 = spec_to_norm.spectral_axis
    y_2 = spec_to_norm.flux

    g_fit = fit_generic_continuum(spec_to_norm)

    y_cont_fitted = g_fit(x_2)
    spec_normalized = spec_to_norm / y_cont_fitted

    test = plt.subplots()[1]  
    test.plot(spec_normalized.spectral_axis, spec_normalized.flux)
    test.grid(True)
    
    return spec_normalized

    
    
def show_emission_lines(spec_normalized):
    #Line fitting with Derivative technique
    lines = find_lines_derivative(spec_norma, flux_threshold=1.2) #check threshold value here--> ??
    print('emission: ', lines[lines['line_type'] == 'emission']) 

    
    
def analyse_line(spec_normalized, line_center_value, around_value):
    sub_region = SpectralRegion((line_center_value-around_value)*u.Angstrom, (line_center_value+around_value)*u.Angstrom)
    sub_spectrum = extract_region(spec_normalized, sub_region)
    spec_temp = Spectrum1D(flux=sub_spectrum.flux,spectral_axis=sub_spectrum.spectral_axis)
    quick_plot_init(spec_temp)
    print(f" ==== Line analysis: {line_center_value} A (+/- {around_value} A) ==== ")
    
    print(estimate_line_parameters(sub_spectrum, models.Lorentz1D()))
    print(" ")
    print(f"GSW : {gaussian_sigma_width(spec_temp)}") #Gaussian Sigma Width
    print(f"FWHM: {fwhm(spec_temp)}")
    print(f"FWZI: {fwzi(spec_temp)} ") #full width at zero intensity
    
    return gaussian_sigma_width(spec_temp), fwhm(spec_temp), fwzi(spec_temp)

   


In [3]:

#####MAIN ANALYSIS ######

#get spectrum with uncertainty 
spec_with_uncertainty = get_uncertainty_from_region(spec,6718, 6824)

#Cut extremity of spectrum
speccut = cut_extremity_spectrum(spec_with_uncertainty, 50, 200)

#Show plot
quick_plot_init(speccut)

print("------------------------")
print("SNR value = ", snr(spec))

#Show continuum
show_continuum(speccut)

#normalize
spec_norma = normalize_spectrum(speccut)

#emmission lines
show_emission_lines(spec_norma)

#show analysys for a line (amplitude, centerline, fwhm)
analyse_line(spec_norma, 6562, 30)








NameError: name 'spec' is not defined